# Using CLDK to generate JUnit tests


In this tutorial, we will use CLDK to generate a JUnit test for all the methods in a Java Application. 

By the end of this tutorial, you will have a JUnit test for all the methods in a Java application. You'll be able to explore some of benefits of using CLDK to perform fast and easy program analysis and build a LLM-based test generator.

You will learn how to do the following:

1. Create a new instance of the CLDK class.
2. Create an analysis object over the Java application.
3. Iterate over all the files in the project.
4. Iterate over all the classes in the file.
5. Iterate over all the methods in the class.
6. Get the code body of the method.
7. Initialize the treesitter utils for the class file content.
8. Sanitize the class for analysis.

Next, we will write a couple of helper methods to:
1.  Format the instruction for the given focal method and class
2.  Prompts the local model on Ollama 
3.  Prints the instruction and LLM output 

## Prequisites 

Before we get started, let's make sure you have the following installed:

1. Python 3.11 or later
2. Ollama 0.3.4 or later
   
We will use ollama to spin up a local granite model that will act as our LLM for this turorial.

### Prerequisite 1: Install ollama

If you don't have ollama installed, please download and install it from here: [Ollama](https://ollama.com/download). 

Once you have ollama, start the server and make sure it is running.

If you're on MacOS, Linux, or WSL, you can check to make sure the server is running by running the following command:


In [ ]:
%%bash
systemctl status ollama

If not, you may have to start the server manually. You can do this by running the following command:

In [ ]:
%%bash
systemctl start ollama

### Prerequisite 2: Pull the latest version of Granite 8b Instruct Model

Once ollama is up and running, you can download the latest version of the Granite 8b Instruct model by running the following command:

In [ ]:
%%bash
ollama pull granite-code:8b-instruct

There are other granite versions available, but for this tutorial, we will use the Granite 8b Instruct model. You if prefer to use a different version, you can replace `8b-instruct` with any of the other [versions](https://ollama.com/library/granite-code/tags).

Let's make sure the model is downloaded by running the following command:

In [ ]:
%%bash
ollama run granite-code:8b-instruct "Write a python function to print 'Hello, World!'"

### Prerequisite 3: Install ollama Python SDK

In [ ]:
%%bash
pip install ollama

### Prerequisite 4: Install CLDK

CLDK is avaliable on github at github.com/IBM/codellm-devkit.git. You can install it by running the following command:

In [ ]:
%%bash
pip install git+https://github.com/IBM/codellm-devkit.git

## Building a JUnit test generator using CLDK and Granite Code Instruct Model

Now that we have all the prerequisites installed, let's start building a JUnit test generator using CLDK and the Granite Code Instruct Model.

### Step 1: Get the sample Java application

For this tutorial, we will use apache commons cli. You can download the source code to a temporary directory by running the following command:

In [ ]:
%%bash
wget https://github.com/apache/commons-cli/archive/refs/tags/rel/commons-cli-1.7.0.zip -O /tmp/commons-cli-1.7.0.zip && unzip -o /tmp/commons-cli-1.7.0.zip -d /tmp 

The project will now be extracted to `/tmp/commons-cli-rel-commons-cli-1.7.0`. We'll remove these files later, so don't worry about the location.

### Step 2: Create a new instance of the CLDK class

Let's import CLDK and create a new analysis object over the Java application:

In [ ]:
from cldk import CLDK


cldk = CLDK(language='java')
java_analysis = cldk.analysis(project_path="/tmp/commons-cli-rel-commons-cli-1.7.0")

Now, let's get all the methods in the project! We will use the `get_methods()` method to get all the methods in the project. This will return dictionary where the key is the class name the values are a map of method signatures to their callable objects.

In [ ]:
all_methods = java_analysis.get_methods()

Now, that we have a dictionary of all the methods in the project, let's write a couple of quick helpers that will prompt the local model on Ollama and print the instruction and LLM output.

In [ ]:
def create_an_prompt(method, reference_type):
    """A simple function to create an instruction for a method"""
    inst = f"Write a summary for the method {method} in the class {reference_type}"
    inst += f"\n```\n{method.code}\n```"
    return inst

In [ ]:
import ollama

def execute_prompt(prompt: str):
    """A simple function to execute a prompt"""
    return ollama.generate(model="granite-code:8b-instruct", prompt=prompt)["response"]

Finally, we will iterate over all the methods in `all_methods` and generate a JUnit test for each method. We will use the `format_instruction` method to format the instruction for the given focal method and class. We will then prompt the local model on Ollama and print the instruction and LLM output with the `prompt_local_model` method.

In [ ]:
import sys

for type_name, method_dict in all_methods.items():
    for method_signature, method in method_dict.items():
        print(f"Method: {method.code}")
        prompt = create_an_prompt(method, type_name)
        response = execute_prompt(prompt)
        print(f"Prompt: {prompt}")
        print(f"response: {response}")        
        